In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "/content/drive/MyDrive/Spring_2023_Project/COMS_572_Source_Code/MNIST"

In [ ]:
## load package
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# load build-in dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
## reshape and normalize data
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
y_train = y_train.reshape(60000,)
y_test = y_test.reshape(10000,)


In [ ]:
y_train_one_hot = tf.keras.utils.to_categorical(y_train)
y_test_one_hot = tf.keras.utils.to_categorical(y_test)

In [ ]:
train_arr_75 = [tf.image.grayscale_to_rgb(tf.image.resize(img, (75, 75))) for img in x_train]
train_arr_75 = np.array(train_arr_75)
train_arr_75.shape

In [ ]:
test_arr_75 = [tf.image.grayscale_to_rgb(tf.image.resize(img, (75, 75))) for img in x_test]
test_arr_75 = np.array(test_arr_75)
test_arr_75.shape

In [ ]:
model = tf.keras.models.load_model('epoch10_stage_2_MNIST_VGG_build_model.h5')
model.summary()

In [ ]:
model.layers[0].trainable=False
model.summary()

In [ ]:
obtained_learning_rate = np.loadtxt('epoch10_stage_1_lr.txt')

In [ ]:
def scheduler(epoch):
  return obtained_learning_rate[epoch]

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      optimizer=opt,
      metrics=[tf.keras.metrics.CategoricalAccuracy()]
  )

In [ ]:
with tf.device('/device:GPU:0'):
  final_history1 = model.fit(train_arr_75, y_train_one_hot, validation_data=(test_arr_75, y_test_one_hot), epochs = 10, verbose=1, callbacks=[callback])

In [ ]:
model.save('epoch10_stage_3_MNIST_VGG_build_model.h5')